In [ ]:
import geopandas as gpd
from numpy.ma.core import append

gpd.read_file('blocks (1).geojson')

In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
from shapely import Polygon, MultiPolygon

from app.gen_planner.python.src import GenPlanner, gen_plan, basic_func_zone

zones, roads = GenPlanner(gpd.read_file('1.gpkg')).poly2func2terr2block(gen_plan)

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from app.gen_planner.python.src import GenPlanner, terr_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2block(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd

zones = {'1': 0.1,
         '2': 0.1,
         '3': 0.1,
         '4': 0.1,
         '5': 0.1,
         '6': 0.1,
         '7': 0.1,
         '8': 0.1, }
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).split_poly(zones, roads_width=50)
zones.explore()

In [ ]:
import geopandas as gpd
import numpy as np
from shapely import MultiPolygon, Polygon

local_crs = 32636
multip = gpd.read_file('2.gpkg').to_crs(local_crs)


def elasatic_wrap(gdf: gpd.GeoDataFrame) -> Polygon:
    gdf = gdf.copy()
    multip = gpd.GeoDataFrame(geometry=gdf.geometry, crs=gdf.crs).explode(ignore_index=True)
    max_dist = np.ceil(multip.apply(lambda row: multip.drop(row.name).distance(row.geometry).min(), axis=1).max(axis=0))
    poly = multip.buffer(max_dist).union_all().buffer(-max_dist)
    if isinstance(poly, MultiPolygon):
        raise RuntimeError('Cant cast Multipolygon to Polygon')
    poly = Polygon(poly.exterior)
    return poly


p = elasatic_wrap(multip)

In [ ]:
gpd.GeoDataFrame(geometry=[p.simplify(250)], crs=local_crs).explore()

In [ ]:
from app.gen_planner.python.src import GenPlanner, gen_plan
import geopandas as gpd

gp = GenPlanner(gpd.read_file('2.gpkg'), rotation=0)
multiterritory = gp.transformed_poly
zones, roads = gp.poly2func2terr2block(gen_plan)

In [ ]:
zones.explore()

# MultiP

In [8]:
import geopandas as gpd
from app.gen_planner.python.src.utils.geom_utils import geometry_to_multilinestring
from shapely.ops import polygonize, unary_union, Polygon


def territory_splitter(gdf_to_split: gpd.GeoDataFrame, splitters: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    local_crs = gdf_to_split.estimate_utm_crs()
    gdf_to_split = gdf_to_split.to_crs(local_crs)
    splitters = splitters.to_crs(local_crs)
    lines_orig = gdf_to_split.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()
    lines_splitters = splitters.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()

    polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_splitters))),
                                crs=local_crs).clip(gdf_to_split.to_crs(local_crs), keep_geom_type=True).explode()

    polygons_points = polygons.copy()
    polygons_points.geometry = polygons.representative_point()
    joined_ind = polygons_points.sjoin(splitters, how='inner', predicate='within').index.tolist()
    polygons['is_splitter'] = polygons.index.isin(joined_ind)
    polygons.geometry = polygons.apply(lambda x: Polygon(x.geometry.exterior) if not x['is_splitter'] else x.geometry,
                                       axis=1)
    non_splitters = polygons[~polygons['is_splitter']]
    contains = non_splitters.sjoin(non_splitters, predicate='contains')
    to_kick = contains[~(contains.index == contains['index_right'])]['index_right'].to_list()
    polygons.drop(index=to_kick, inplace=True)
    return polygons


local_crs = 32636

territory = gpd.read_file('project ШлиссельбургMulti.geojson').to_crs(local_crs)
roads = gpd.read_file('roads_shlis.geojson').to_crs(local_crs)
# roads.geometry = roads.geometry.buffer(5, resolution=2)

multipolys = territory_splitter(territory, roads)
multipolys.explore(style_kwds={'fillOpacity': 0.5})

In [9]:
from app.gen_planner.python.src import func_zones, GenPlanner

multipolys = multipolys[~multipolys['is_splitter']]
zones, roads = GenPlanner(multipolys).poly2terr2block(func_zones.residential_func_zone)

[                                             geometry  roads_width
0   LINESTRING (388637.133 6643484.305, 388613.551...           10
1   LINESTRING (389069.248 6643906.794, 388621.209...           10
2   LINESTRING (388621.209 6643887.317, 388620.019...           10
3   LINESTRING (388620.019 6643886.165, 388617.392...           10
4   LINESTRING (388617.392 6643495.859, 388637.133...           10
5   LINESTRING (388400.669 6644312.182, 388398.112...           10
6   LINESTRING (388398.112 6644002.811, 388620.019...           10
7   LINESTRING (388718.85 6644310.466, 388713.63 6...           10
8   LINESTRING (388713.63 6644313.775, 388400.669 ...           10
9   LINESTRING (389069.248 6643906.794, 389068.963...           10
10  LINESTRING (388027.079 6644312.219, 388391.439...           10
11  LINESTRING (388391.439 6644318.716, 388396.449...           10
12  LINESTRING (389682.695 6644809.792, 389682.611...           10
13  LINESTRING (389135.387 6644314.911, 389133.804...        

In [11]:
from app.gen_planner.python.src import func_zones, GenPlanner,terr_zones
zones2, roads2 = GenPlanner(gpd.GeoDataFrame(geometry =[multipolys.loc[0].geometry],crs=32636)).poly2block(terr_zone=terr_zones.residential_terr)

[Empty GeoDataFrame
Columns: []
Index: [],                                              geometry             road_lvl  \
0   LINESTRING (389970.46 6643300.903, 389940.06 6...  local road, level 1   
1   LINESTRING (389940.06 6643684.223, 389455.148 ...  local road, level 1   
2   LINESTRING (388711.479 6643687.126, 389058.894...  local road, level 1   
3   LINESTRING (390059.757 6644207.693, 390058.575...  local road, level 1   
4   LINESTRING (389455.148 6643685.801, 389454.424...  local road, level 1   
5   LINESTRING (389454.424 6643686.563, 389059.956...  local road, level 1   
6   LINESTRING (390058.575 6643877.694, 390057.541...  local road, level 1   
7   LINESTRING (390057.541 6643877.108, 389940.06 ...  local road, level 1   
8   LINESTRING (389970.46 6643300.903, 389972.778 ...  local road, level 1   
9   LINESTRING (389058.894 6643687.254, 389059.956...  local road, level 1   
10  LINESTRING (390123.007 6643008.682, 389972.778...  local road, level 1   

    roads_width  
0 

In [25]:
m1 = multipolys.reset_index().explore( style_kwds={'fillOpacity': 0.2})
zones.explore(m=m1,column='zone_name', tiles='CartoDB positron', cmap='Dark2', categorical=True,style_kwds={'fillOpacity': 0.8})
zones2.explore(m=m1)
m1

In [71]:
local_crs = 32636

lines_orig = multipolys.geometry.apply(geometry_to_multilinestring).to_list()
lines_new = zones.geometry.apply(geometry_to_multilinestring).to_list()
polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_new))),
                            crs=local_crs).explode()
polygons['subsplit_area'] = polygons.area
polygons_points = polygons.copy()
polygons_points.geometry = polygons_points.representative_point()
polygons_points = polygons_points.sjoin(zones, how='inner', predicate='within').drop(columns='index_right')
multipolys['orig_area'] = multipolys.area
division = multipolys.sjoin(polygons_points, how='inner', predicate='contains').drop(
    columns=['index_right', 'is_splitter', 'ratio'])

division['zone_str'] = division['zone_name'].apply(lambda x: x.name)
division = division.reset_index().groupby(['index', 'zone_str']).agg(
    {'subsplit_area': 'sum', 'orig_area': 'first', 'zone_name': 'first', 'geometry': 'first'}).reset_index().set_index(
    'index')
division

,zone_str,subsplit_area,orig_area,zone_name,geometry
index,,,,,
0,business,7.566879e+04,2.233466e+06,"Territory zone ""business""","POLYGON ((389074.006 6643434.599, 389114.274 6..."
0,residential,2.148560e+06,2.233466e+06,"Territory zone ""residential""","POLYGON ((389074.006 6643434.599, 389114.274 6..."
1,business,1.795511e+03,1.795511e+03,"Territory zone ""business""","POLYGON ((389074.966 6643470.945, 389076.347 6..."
3,business,3.034020e+03,3.034020e+03,"Territory zone ""business""","POLYGON ((389077.501 6643566.852, 389078.195 6..."
6,residential,1.617271e+05,3.997062e+05,"Territory zone ""residential""","POLYGON ((388503.763 6644217.97, 388505.102 66..."
6,special,1.023423e+05,3.997062e+05,"Territory zone ""special""","POLYGON ((388503.763 6644217.97, 388505.102 66..."
6,transport,1.248684e+05,3.997062e+05,"Territory zone ""transport""","POLYGON ((388503.763 6644217.97, 388505.102 66..."
8,special,1.764760e+05,5.011487e+05,"Territory zone ""special""","POLYGON ((387760.992 6644473.129, 387794.625 6..."
8,transport,3.186859e+05,5.011487e+05,"Territory zone ""transport""","POLYGON ((387760.992 6644473.129, 387794.625 6..."


division['subsplit_area']  - площадь по каждой тз в поллигоне

In [120]:
import pandas as pd

target_zones = pd.DataFrame.from_dict(func_zones.residential_func_zone.zones_ratio, orient='index',
                                      columns=['zones_ratio']).reset_index().rename(columns={'index': 'zone'})
target_zones['zones_ratio'] = target_zones['zones_ratio'] / target_zones['zones_ratio'].sum()
target_zones['target_area'] = target_zones['zones_ratio'] * division['subsplit_area'].sum()
target_zones['min_area'] = target_zones['zone'].apply(lambda x: x.min_block_area)
target_zones['zone'] = target_zones['zone'].apply(lambda x: x.name)
target_zones.set_index('zone', inplace=True)
target_zones['area_last'] = target_zones['target_area']
target_zones

,zones_ratio,target_area,min_area,area_last
zone,,,,
residential,0.555556,3.068578e+06,160000,3.068578e+06
business,0.111111,6.137156e+05,160000,6.137156e+05
recreation,0.111111,6.137156e+05,320000,6.137156e+05
transport,0.111111,6.137156e+05,160000,6.137156e+05
agriculture,0.055556,3.068578e+05,640000,3.068578e+05
special,0.055556,3.068578e+05,160000,3.068578e+05


In [121]:
possible_areas = division.reset_index().groupby('index').agg({'zone_str': list, 'orig_area': 'first'})
possible_areas['area_last'] = possible_areas['orig_area']
possible_areas = possible_areas.join(
    possible_areas.apply(lambda x: pd.Series([0 for y in x.zone_str], index=x.zone_str), axis=1))
possible_areas = possible_areas.drop(columns=['zone_str'])
possible_areas

,orig_area,area_last,agriculture,business,recreation,residential,special,transport
index,,,,,,,,
0,2.233466e+06,2.233466e+06,NaN,0.0,NaN,0.0,NaN,NaN
1,1.795511e+03,1.795511e+03,NaN,0.0,NaN,NaN,NaN,NaN
3,3.034020e+03,3.034020e+03,NaN,0.0,NaN,NaN,NaN,NaN
6,3.997062e+05,3.997062e+05,NaN,NaN,NaN,0.0,0.0,0.0
8,5.011487e+05,5.011487e+05,NaN,NaN,NaN,NaN,0.0,0.0
10,1.305045e+03,1.305045e+03,NaN,NaN,NaN,NaN,NaN,0.0
11,1.923937e+03,1.923937e+03,NaN,NaN,NaN,NaN,NaN,0.0
14,1.130650e+03,1.130650e+03,NaN,NaN,NaN,NaN,0.0,NaN
16,4.682711e+03,4.682711e+03,NaN,NaN,NaN,NaN,0.0,NaN


In [122]:
while target_zones['area_last'].sum() > 0:
    zone_to_search = target_zones['area_last'].idxmax()
    area_last = target_zones.loc[zone_to_search, 'area_last']
    print('area_last ', area_last)
    print('zone_to_search', zone_to_search)
    block_area = target_zones.loc[zone_to_search, 'min_area']
    
    possible_areas_gt0 = possible_areas[possible_areas['area_last']>0]
    zone_to_add_id = possible_areas_gt0[~possible_areas_gt0[zone_to_search].isna()]['area_last'].idxmax()
    
    area_to_fill = min(possible_areas.loc[zone_to_add_id, 'area_last'], block_area, area_last)
    print('area_to_fill', area_to_fill)
    possible_areas.at[zone_to_add_id, 'area_last'] = possible_areas.loc[zone_to_add_id, 'area_last'] - area_to_fill
    possible_areas.at[zone_to_add_id, zone_to_search] = possible_areas.at[zone_to_add_id, zone_to_search]+area_to_fill
    target_zones.at[zone_to_search, 'area_last'] = target_zones.loc[zone_to_search, 'area_last'] - area_to_fill
    print(target_zones['area_last'].sum())
    print('-----')



area_last  3068577.9186151726
zone_to_search residential
area_to_fill 160000
5363440.253507311
-----
area_last  2908577.9186151726
zone_to_search residential
area_to_fill 160000
5203440.253507311
-----
area_last  2748577.9186151726
zone_to_search residential
area_to_fill 160000
5043440.253507311
-----
area_last  2588577.9186151726
zone_to_search residential
area_to_fill 160000
4883440.253507311
-----
area_last  2428577.9186151726
zone_to_search residential
area_to_fill 160000
4723440.253507311
-----
area_last  2268577.9186151726
zone_to_search residential
area_to_fill 160000
4563440.253507311
-----
area_last  2108577.9186151726
zone_to_search residential
area_to_fill 160000
4403440.253507311
-----
area_last  1948577.9186151726
zone_to_search residential
area_to_fill 160000
4243440.253507311
-----
area_last  1788577.9186151726
zone_to_search residential
area_to_fill 160000
4083440.2535073115
-----
area_last  1628577.9186151726
zone_to_search residential
area_to_fill 160000
3923440.25350

ValueError: attempt to get argmax of an empty sequence

In [123]:
possible_areas

,orig_area,area_last,agriculture,business,recreation,residential,special,transport
index,,,,,,,,
0,2.233466e+06,0.000000,NaN,153466.249186,NaN,2080000.0,NaN,NaN
1,1.795511e+03,1795.510785,NaN,0.000000,NaN,NaN,NaN,NaN
3,3.034020e+03,3034.020060,NaN,0.000000,NaN,NaN,NaN,NaN
6,3.997062e+05,0.000000,NaN,NaN,NaN,320000.0,0.0,79706.243180
8,5.011487e+05,0.000000,NaN,NaN,NaN,NaN,160000.0,341148.749781
10,1.305045e+03,0.000000,NaN,NaN,NaN,NaN,NaN,1305.044987
11,1.923937e+03,0.000000,NaN,NaN,NaN,NaN,NaN,1923.937427
14,1.130650e+03,1130.649734,NaN,NaN,NaN,NaN,0.0,NaN
16,4.682711e+03,4682.711448,NaN,NaN,NaN,NaN,0.0,NaN


In [124]:
target_zones

,zones_ratio,target_area,min_area,area_last
zone,,,,
residential,0.555556,3.068578e+06,160000,188577.918615
business,0.111111,6.137156e+05,160000,140249.334537
recreation,0.111111,6.137156e+05,320000,162091.820483
transport,0.111111,6.137156e+05,160000,189631.608349
agriculture,0.055556,3.068578e+05,640000,169876.493699
special,0.055556,3.068578e+05,160000,146857.791862


In [ ]:

from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr(func_zones.residential_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)